In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import os
import cv2
from skimage.feature import greycomatrix, greycoprops, local_binary_pattern

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
path = '/content/drive/My Drive/data'

Mounted at /content/drive


In [ ]:
DATADIR = path
CATEGORIES = []
def create_category():
        for dir in os.listdir(os.path.join(DATADIR,'train')):
              CATEGORIES.append(dir)
create_category()

In [26]:
def extractFeatures(img):
    grey = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
#     lbp = local_binary_pattern(gray, 8, 1)
#     lbp = np.float32(lbp)
#     hist = cv2.calcHist([lbp],[0],None,[256],[0,256])
#     hist = hist/(331*331)
#     hist = np.array(hist).flatten()
    
    glcm = greycomatrix(grey, distances=[1, 3, 5], angles=[0, np.pi/4, np.pi/2], levels=256, symmetric=True, normed=True)
    haralick = []    
    haralick.append(greycoprops(glcm, 'dissimilarity'))
    haralick.append(greycoprops(glcm, 'correlation'))
    haralick.append(greycoprops(glcm, 'energy'))
    haralick.append(greycoprops(glcm, 'contrast'))
    haralick.append(greycoprops(glcm, 'homogeneity'))
    haralick.append(greycoprops(glcm, 'ASM'))
    haralick = np.array(haralick).flatten()

    stats = haralick
    
    return stats
    

In [ ]:
#Đọc dữ liệu ảnh trong folder train
def create_training_data():
    training_data = []
    for category in CATEGORIES:
        path = os.path.join(DATADIR, 'train', category)
        class_num = CATEGORIES.index(category)
        
        for img in os.listdir(path):            
            img_array = cv2.imread(os.path.join(path,img))
            training_data.append([img_array, class_num])
    return np.array(training_data, dtype=object)
training_data = create_training_data()

In [27]:
#Trích chọn các đặc trưng và gán label tương ứng cho dữ liệu train
x = []
y = []
for img, label in (training_data):
    stats = extractFeatures(img)
    
    x.append(stats)
    y.append(label)
    
x_train=np.float32(x)
y_train=np.float32(y)


In [28]:
# RGB_img = cv2.cvtColor(training_data[900][0], cv2.COLOR_BGR2RGB)
# plt.imshow(RGB_img)
# print(training_data[900][1])
# print(CATEGORIES[4])

# grayImg = cv2.cvtColor(training_data[0][0], cv2.COLOR_BGR2GRAY)

# lbp1 = local_binary_pattern(grayImg, 8, 1)
# lbp1 = np.float32(lbp1)
# hist = cv2.calcHist([lbp1],[0],None,[256],[0,256])


x_train.shape

#plt.imshow(grayImg,cmap='gray')

#x_train[0]


(4351, 54)

In [29]:
#Đọc dữ liệu ảnh trong folder valid
def create_test_data():
    test_data = []
    for category in CATEGORIES:
        path = os.path.join(DATADIR, 'valid', category)
        class_num = CATEGORIES.index(category)
        for img in os.listdir(path):
            img_array = cv2.imread(os.path.join(path,img))
            test_data.append([img_array, class_num])
    return np.array(test_data, dtype=object)
test_data = create_test_data()

In [30]:
#Trích chọn các đặc trưng và gán label tương ứng cho dữ liệu valid
x_valid = []
y_valid = []
for img, label in test_data:
    stats = extractFeatures(img)

    x_valid.append(stats)
    y_valid.append(label)

x_test=np.float32(x_valid)
y_test=np.float32(y_valid)


In [31]:
knn=cv2.ml.KNearest_create()
knn.train(x_train,cv2.ml.ROW_SAMPLE,y_train)

True

In [32]:
result=knn.findNearest(x_test, 3)

In [33]:
correct = np.count_nonzero(result[1].flatten() == y_test)
accuracy = correct*100.0/len(y_test)
accuracy

71.29032258064517

In [ ]:
# stats = extractFeatures(test_data[900][0])
# stats = np.float32([stats])
# test = knn.findNearest(stats, 3)
# print(test[1])
# print(y_test[900])


[[4.]]
4.0
